# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig

from azureml.train.hyperdrive.parameter_expressions import normal, choice

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import joblib

from train import clean_data, get_dataset

from sklearn.model_selection import train_test_split
import pandas as pd

import capstone_constants as c_constants

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Detailed notes provided in the project file automl.ipynb

In [ ]:
ws = Workspace.from_config()

experiment=Experiment(ws, c_constants.HYPER_EXPERIMENT_NAME)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#

# See if it already exists
# this ofcourse helps if you are reunning the notebook from teh start and do not
# need to recreate the compute cluster
try:
    compute_cluster = ComputeTarget(ws, c_constants.COMPUTE_CLUSTER_HYPER)
    # No exception thrown - Found it - use it below
except ComputeTargetException:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance - from the Microsoft recommended options
    cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS3_v2',
                                                            max_nodes=4,
                                                            description='CPU Compute Cluster created programatically')

    # Next, create the cluster
    compute_cluster = ComputeTarget.create(ws, c_constants.COMPUTE_CLUSTER_HYPER, cluster_config)

# We have a compute cluster - either newly created - or created earlier
# We may wait for the create operation to complete
compute_cluster.wait_for_completion(show_output=True)

Workspace name: quick-starts-ws-161266
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-161266
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
# Note that when creatingt he dataset, the ouput is changed from a quality score (1 - 10)
# to a binary 1/0 where 1 is applied to all entried where the quality score is 7 or greater
# The intent here is to define a dataset where the label is "desired" (or not) and not a
# quality score

train_ds = getdataset()

In [ ]:

# Take a peek at the data by converting the same to a Pandas dataframe
train_df = train_ds.to_pandas_dataframe()
    
# print the data
train_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,2.186612,0.315085,3.270609,1.569104,-0.081122,-0.488506,1.238629,2.040392,-1.511694,3.418622,-1.474270,0
1,-0.538932,-0.478608,-0.034635,-1.033390,-0.081122,-1.164692,1.862187,-0.677847,0.938628,-0.436771,0.394706,0
2,1.357099,0.513509,0.130627,0.711464,-0.493060,-0.782500,-0.714401,0.181424,-0.452095,-1.137752,0.882265,0
3,-1.842453,1.902473,-1.935150,-1.003816,-0.447289,-1.429287,-1.514437,-1.072376,1.932001,0.439455,0.557225,0
4,-0.301928,-0.180973,1.865881,1.776120,0.330815,-0.194512,0.062105,1.361668,-0.518320,0.001342,-0.986711,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3913,0.764589,1.406414,-1.191470,0.100272,-0.218435,1.569454,0.273879,0.351941,0.276379,-1.050129,-0.417892,0
3914,1.475600,-1.569937,0.378521,-1.082680,-0.538831,0.040684,-1.232072,-1.259610,-2.571292,0.527077,1.207304,0
3915,2.068110,0.910356,3.022716,1.135355,-0.309976,-0.841299,0.462123,1.428538,-1.246794,2.104284,-0.986711,0
3916,-1.486947,0.414297,-0.034635,0.198851,-0.401518,-0.606103,-1.020297,-0.450491,0.408829,0.001342,0.475966,0


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The train.py script uses [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression) from scikit-learn. The task is to maximize accuracy. 

### The parameterts to the model are:
Inverse of regularization strength : --C<br/>
Max Iterations : --max_iter

A random combination of values for these parameters will be iterated over to determine the optimal values for these two hyper parameters.
<br/><br/>

[Bandit Policy](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py) defines an early termination policy based on slack criteria (slack-factor)
<br/><br/>
The primary metric is accuracy, inferred by scikit-learn LogisticRegression


In [ ]:
# BanditPolicy is a type of EarlyTerminationPolicy
# More info can be found at 
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# from train.py - we know that the parameters are --C and --max_iter
# train.py uses the scikit-learn LogisticRegression model

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(50, 100, 200, 300, 400),
})

# We need to specify/provide an environment
# We could create our own environment with corresponding yaml file
# We may also provide an environment 'out-of-the-box' from one of the many environments provided

curated_env = Environment.get(workspace=ws, name=CURATED_ENV_NAME)

# SKLearn is deprecated - instead we specity the run configuration using ScriptRunConfig
# It is necessary to pass the directory, the python script, the compute cluster and the environment

#TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>


run_config = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=compute_cluster,
    environment=curated_env,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
)

In [ ]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(config=hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_5d9d47e2-31cb-4c09-9272-dd0b36430735
Web View: https://ml.azure.com/runs/HD_5d9d47e2-31cb-4c09-9272-dd0b36430735?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-161266/workspaces/quick-starts-ws-161266&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-18T02:03:48.077267][API][INFO]Experiment created<END>\n""<START>[2021-10-18T02:03:48.558308][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2021-10-18T02:03:48.857842][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:

assert(hyperdrive_run.get_status() == "Completed")


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# Get your best run and save the model from that run.


### YOUR CODE HERE ###
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

# Get the model hyperparameter values corresponding to the best run
print(f'********** {best_hyperdrive_run.get_details()["runDefinition"]["arguments"]}')
# print(best_hyperdrive_run.get_details())

# Get the accuracy corresponsding to the best run
print(f'********** Best HyperDrive run Accuracy: {best_hyperdrive_run.get_metrics()["Accuracy"]}')

********** ['--C', '0.01', '--max_iter', '50']
********** Best HyperDrive run Accuracy: 0.8295918367346938


In [ ]:
#TODO: Save the best model
# Save and download the model
print(f'The file name for the model is: {best_hyperdrive_run.get_file_names()[-1]}')

# download the file in the notebook context. In this case, under Users/**user**/outputs/
best_hyperdrive_run.download_file(name=best_hyperdrive_run.get_file_names()[-1], output_file_path=c_constants.DEPLOYED_HYPER_MODEL_PATH)

The file name for the model is: user_logs/std_log.txt


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
model = best_hyperdrive_run.register_model(model_name=c_constants.DEPLOYED_HYPER_MODEL_NAME, 
                                           model_path = c_constants.DEPLOYED_HYPER_MODEL_PATH)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/best_run_hyperdrive.pkl in the set of files uploaded to the run: ['logs/azureml/16_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/best_run_hyperdrive.pkl in the set of files uploaded to the run: ['logs/azureml/16_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'sy

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

